In [1]:
import openrouteservice
from bs4 import BeautifulSoup
import requests
import unicodedata
from csv import writer
import re
import pandas as pd
import numpy as np
import json


#spark session
from pyspark.sql import SparkSession
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql import functions as F


import os
import re 
import time

#folium
import geopandas as gpd
import re
import folium

In [2]:
'''

calculate centroid for each postcode

'''

# get postcode list
domain = pd.read_csv('../../data/raw/domain_preprocessed.csv')
postcode_list = list(domain['postcode'].unique()) 


# filter postcode based on properties
selected_columns = ['POSTCODE','geometry']
postcode_df = gpd.read_file("../../data/raw/POSTCODE/POSTCODE_POLYGON.shp")[selected_columns]
postcode_df['POSTCODE'] = pd.to_numeric(postcode_df['POSTCODE'])

# remove postcode not in domain postcode list
postcode_df= postcode_df.loc[postcode_df['POSTCODE'].isin(postcode_list)].reset_index()
postcode_df = postcode_df.drop(columns = 'index')

# add centroid: (y, x) since we want (lat, long)
postcode_df['centroid'] = postcode_df['geometry'].apply(lambda x: (x.centroid.x, x.centroid.y))

# find CBD centroid
CBD_centroid = postcode_df.loc[postcode_df['POSTCODE'] == 3000]['centroid'].values[0]




In [3]:
# output postcode with their centroid
filename = 'postcode_centroid.csv'
output_dir = '../../data/raw/external-data/'

output_dir_full = f'{output_dir}{filename}'
postcode_df.to_csv(output_dir_full)

In [4]:
# use your own key
client = openrouteservice.Client(key='5b3ce3597851110001cf6248f596dd33da864705b4185662da1dd01d')

# dic to store info 
dic_postcode = {'postcode':[],'distance':[], 'duration':[]}

In [5]:
"""

calculate distance from postcode centroid to CBD(3000) centroid

"""

def calcualte_centroid_to_CBD(postcode_list):
    count = 0
    for postcode in postcode_list:
        if count/10 == 0:
            time.sleep(5)
        postcode_centroid = postcode_df[postcode_df['POSTCODE'] == postcode]['centroid'].values[0]
        route = client.directions(
                coordinates= [postcode_centroid, CBD_centroid],
                profile='driving-car',
                format='geojson',
                radiuses = 1000,
            )
        distance = route['features'][0]['properties']['segments'][0]['distance']
        duration = route['features'][0]['properties']['segments'][0]['duration']
        dic_postcode['postcode'].append(postcode)
        dic_postcode['distance'].append(distance)
        dic_postcode['duration'].append(duration)
        count += 1

In [6]:
# recursively find distance between centroid and CBD
for i in range(0,180,20):
    calcualte_centroid_to_CBD(postcode_list[i:i+20])
    print(f"round{i} finished")
    time.sleep(10)
    
# convert dictionary to dataframe to output csv
centroid_to_CBD_df = pd.DataFrame.from_dict(dic_postcode)

round0 finished
round20 finished


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate l

round40 finished


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate l

round60 finished


...

round120 finished


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate l

round140 finished


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate l

round160 finished


In [7]:
# output distance and duration to CBD centroid for each postcode 
# output to external-data
filename = 'centroid_to_CBD.csv'
output_dir = '../../data/raw/ors-data/'
centroid_to_CBD_df = pd.DataFrame.from_dict(dic_postcode)
output_dir_full = f'{output_dir}{filename}'
centroid_to_CBD_df.to_csv(output_dir_full)